In [45]:
import re
import pandas as pd
import os

In [46]:
df = pd.read_csv('totalset.csv', compression = 'gzip')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [47]:
# df_1 = : 34807
# df_2 = 34808 tm 36374 
# df_3 = 36375 tm 40374
# df_4 = 40375 tm 40593
# df_5 = 40594 :

df_1 = df[:34808]
df_2 = df[34808:36375] 
df_3 = df[36375:40375]
df_4 = df[40375:40594]
df_5 = df[40594:]

In [48]:
print(df_5['identifier'].head())
print(df_5['identifier'].tail())

40594    537184
40595    537185
40596    537186
40597    537187
40598    537188
Name: identifier, dtype: object
40793    537384
40794    537385
40795    537386
40796    537387
40797    537388
Name: identifier, dtype: object


In [49]:
# first_dataset = 0 - 34807, len = 34808 (unique: 6068) = Meertens
#df_1 = df[:34808]
# second_dataset = 34808 - 36374, len = 1567 = Impact Books
#df_2 = df[34808:36375]
# third_dataset = 36375 - 40374, len = 4000 (_ocr: 2000) = newspaper pages
#df_3 = df[36375:40375-dropped]
# fourt_dataset = 40375 - 40593, len = 219 = DBNL books
#df_4 = df[40375-dropped:40594-dropped]
# fifth_dataset = 40594 - 40797, len = 204 = radiobulletins
#df_5 = df[40594-dropped:]

print('df_1:', str(df_1['identifier'].head(1))) # Meertens
print('df_2:', str(df_2['identifier'].head(1))) # Impact Books
print('df_3:', str(df_3['identifier'].head(1))) # newspaper pages
print('df_4:', str(df_4['identifier'].head(1))) # DBNL books
print('df_5:', str(df_5['identifier'].head(1))) # radiobulletins

df_1: 0    ddd:010728187:mpeg21:a0005
Name: identifier, dtype: object
df_2: 34808    538813
Name: identifier, dtype: object
df_3: 36375    DDD_010197037_001.jp2_ocr
Name: identifier, dtype: object
df_4: 40375    _vad003182501
Name: identifier, dtype: object
df_5: 40594    537184
Name: identifier, dtype: object


In [50]:
# for dataset 1: everything is 1600s
meta_data2 = pd.read_excel('Metadata_IMPACT_books.xlsx')
meta_data3 = pd.read_excel('MetadataHistorical_newspaper_OCR_groundtruth (2).xlsx') 
meta_data4 = pd.read_excel('Metadata_DBNL_OCR_v1.xlsx')
# for dataset 5: everything is 1900s

In [51]:
print(meta_data2.columns)
print(meta_data3.columns)
print(meta_data4.columns)

Index(['Did', 'Dtitle', 'Dauthor', 'Dyear', 'Dmonth', 'Dday', 'Doriginal'], dtype='object')
Index(['Identifier', 'Date of publication', 'Date of production', 'Producer',
       'ABBYY Version', 'Title', 'Page number', 'Microfilm'],
      dtype='object')
Index(['ti_id', 'titel', 'jaar', 'druk'], dtype='object')


In [52]:
print('meta_data2:', str(meta_data2['Did'].head(1))) # Impact Books
print('meta_data3:', str(meta_data3['Identifier'].head(1))) # newspaper pages
print('meta_data4:', str(meta_data4['ti_id'].head(1))) # DBNL books

meta_data2: 0    529524
Name: Did, dtype: int64
meta_data3: 0    ddd:011019417:mpeg21
Name: Identifier, dtype: object
meta_data4: 0    _vad003177601
Name: ti_id, dtype: object


In [53]:
#datasets with constant values for year
df_1_year = df_1
df_5_year = df_5
df_1_year['year'] = '16--'
df_5_year['year'] = '1937'

C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [54]:
print(meta_data2.head(1))

      Did                                             Dtitle    Dauthor  \
0  529524  Rechtsgeleerd advis in de zaak van den gewezen...  B. Voorda   

   Dyear  Dmonth  Dday                    Doriginal  
0   1796     NaN   NaN  113/dpo_113_0003_master.tif  


In [55]:
meta_data2['year'] = meta_data2['Dyear']
meta_data2['identifier'] = meta_data2['Did'].astype(str)
meta_data2 = meta_data2[['identifier', 'year']]

In [56]:
df_2_year = pd.merge(left=df_2, right=meta_data2, on='identifier')
df_2_year.head(1)

,identifier,gt text,ocr text,CER,WER,WER (order independent),dictionary lookup gt,dictionary lookup ocr,jaccard_coefficient,levenshtein_dist_normalized,source,word count gt,word count ocr,year
0,538813,de Gebeurtenisſen in I787 enz. voorgevallen. I...,"de Gebeurtenis/en in 1787 enz, voorgevallen, i...",5.2,21.02,20.17,84.82,80.73,0.58,0.07,books,342,330,1791


In [57]:
print(meta_data3.head(1))

             Identifier  Date of publication Date of production Producer  \
0  ddd:011019417:mpeg21  1990-08-16 00:00:00         2012-05-23      CCS   

   ABBYY Version                       Title  Page number  Microfilm  
0            8.1  Nieuwsblad van het Noorden            1        NaN  


In [58]:
print(meta_data3.columns)

Index(['Identifier', 'Date of publication', 'Date of production', 'Producer',
       'ABBYY Version', 'Title', 'Page number', 'Microfilm'],
      dtype='object')


In [59]:
df_3.head()

,identifier,gt text,ocr text,CER,WER,WER (order independent),dictionary lookup gt,dictionary lookup ocr,jaccard_coefficient,levenshtein_dist_normalized,source,word count gt,word count ocr
36375,DDD_010197037_001.jp2_ocr,No. 9922. Tweede Blad. Vrijdag 29 Juli 1910. R...,JSo. 9822 xTweede Blad. Vrijdag 29 Juli 1910. ...,12.93,29.10,23.41,89.44,75.36,0.481129,0.130480,newspapers,4337,4335
36376,DDD_011219339_001.jp2_ocr,De Surinamer Redactie J. WILLEBRANDS Dagblad v...,De Surinamer •v* WILLEBRANDS:i^ Dagblad voor S...,27.15,39.65,33.92,90.90,66.95,0.355100,0.264095,newspapers,2487,2467
36377,DDD_011179281_001.jp2_ocr,"DE GRONDWET. Mrs. L. Mulder, Uitgeefster. JAAR...","DE GRONDWET. Mrs. 3L.. Mulder, TJitgêelster. r...",4.26,13.85,12.32,86.93,78.46,0.677461,0.044412,newspapers,3725,3755
36378,DDD_010948945_001.jp2_ocr,Het Vrije Volk Maandag 25 augustus 1986 42e ja...,Het Vrije Volk Maandag 25 augustus 1986 42e ja...,8.00,10.05,7.62,90.13,85.08,0.690794,0.079328,newspapers,2104,2169
36379,MMGARO01_000177181_002_master.jp2_ocr,"De troepen vertrokken 8 en 9 Sept, uit Tientsi...",De tioepen vertrokken 8 en 9 Sept. uit Tientsi...,0.83,2.94,2.69,90.05,87.96,0.900431,0.008424,newspapers,3576,3595


In [60]:
# df_3: DDD_010197037_001.jp2_ocr
# df_3: MMGARO01_000177181_002_master.jp2_ocr
# metadata3: ddd:010860580:mpeg21
# metadata3: MMKB08:000192440:mpeg21


In [61]:
df_3= df_3[~df_3.identifier.str.contains("_reocr")].reset_index(drop = True)

In [62]:
len(df_3)

2000

In [63]:
def shorten(identifier):
    uniform = identifier.replace('_', ':')
    splitted = uniform.split(':')
    new_id = (':'.join(splitted[:2])).lower()
    return new_id 

df_3['short id'] = df_3['identifier'].apply(lambda x: shorten(x))
meta_data3['short id'] = meta_data3['Identifier'].apply(lambda x: shorten(x))


# df_3: DDD_010197037_001.jp2_ocr
# df_3: MMGARO01_000177181_002_master.jp2_ocr
# metadata3: ddd:010860580:mpeg21
# metadata3: MMKB08:000192440:mpeg21

In [64]:
meta_data3.head()

,Identifier,Date of publication,Date of production,Producer,ABBYY Version,Title,Page number,Microfilm,short id
0,ddd:011019417:mpeg21,1990-08-16 00:00:00,2012-05-23,CCS,8.1,Nieuwsblad van het Noorden,1,NaN,ddd:011019417
1,ddd:010535134:mpeg21,1932,NaT,CCS,8.1,NaN,2,NaN,ddd:010535134
2,ddd:010569784:mpeg21,1985-12-30 00:00:00,2010-03-16,CCS,8.1,Nederlands dagblad : gereformeerd gezinsblad /...,1,NaN,ddd:010569784
3,ddd:010674667:mpeg21,1930,NaT,CCS,8.1,NaN,1,NaN,ddd:010674667
4,ddd:010371904:mpeg21,1963-01-17 00:00:00,2010-04-09,CCS,8.1,De waarheid,2,NaN,ddd:010371904


In [65]:
df_3.head()

,identifier,gt text,ocr text,CER,WER,WER (order independent),dictionary lookup gt,dictionary lookup ocr,jaccard_coefficient,levenshtein_dist_normalized,source,word count gt,word count ocr,short id
0,DDD_010197037_001.jp2_ocr,No. 9922. Tweede Blad. Vrijdag 29 Juli 1910. R...,JSo. 9822 xTweede Blad. Vrijdag 29 Juli 1910. ...,12.93,29.10,23.41,89.44,75.36,0.481129,0.130480,newspapers,4337,4335,ddd:010197037
1,DDD_011219339_001.jp2_ocr,De Surinamer Redactie J. WILLEBRANDS Dagblad v...,De Surinamer •v* WILLEBRANDS:i^ Dagblad voor S...,27.15,39.65,33.92,90.90,66.95,0.355100,0.264095,newspapers,2487,2467,ddd:011219339
2,DDD_011179281_001.jp2_ocr,"DE GRONDWET. Mrs. L. Mulder, Uitgeefster. JAAR...","DE GRONDWET. Mrs. 3L.. Mulder, TJitgêelster. r...",4.26,13.85,12.32,86.93,78.46,0.677461,0.044412,newspapers,3725,3755,ddd:011179281
3,DDD_010948945_001.jp2_ocr,Het Vrije Volk Maandag 25 augustus 1986 42e ja...,Het Vrije Volk Maandag 25 augustus 1986 42e ja...,8.00,10.05,7.62,90.13,85.08,0.690794,0.079328,newspapers,2104,2169,ddd:010948945
4,MMGARO01_000177181_002_master.jp2_ocr,"De troepen vertrokken 8 en 9 Sept, uit Tientsi...",De tioepen vertrokken 8 en 9 Sept. uit Tientsi...,0.83,2.94,2.69,90.05,87.96,0.900431,0.008424,newspapers,3576,3595,mmgaro01:000177181


In [66]:
meta_data3['year'] = meta_data3['Date of publication']
meta_data3['identifier'] = meta_data3['Identifier']
meta_data3 = meta_data3[['short id', 'year']]

In [67]:
df_3_year = pd.merge(left=df_3, right=meta_data3, on='short id')
df_3_year = df_3_year.drop("short id", axis=1)
df_3_year.head(1)


,identifier,gt text,ocr text,CER,WER,WER (order independent),dictionary lookup gt,dictionary lookup ocr,jaccard_coefficient,levenshtein_dist_normalized,source,word count gt,word count ocr,year
0,DDD_010197037_001.jp2_ocr,No. 9922. Tweede Blad. Vrijdag 29 Juli 1910. R...,JSo. 9822 xTweede Blad. Vrijdag 29 Juli 1910. ...,12.93,29.1,23.41,89.44,75.36,0.481129,0.13048,newspapers,4337,4335,1910


In [68]:
len(df_3_year)

2000

In [69]:
df_4.head(1)

,identifier,gt text,ocr text,CER,WER,WER (order independent),dictionary lookup gt,dictionary lookup ocr,jaccard_coefficient,levenshtein_dist_normalized,source,word count gt,word count ocr
40375,_vad003182501,Boekbeschouwing. De gemoedsgesteldheid van Dav...,"SEM1N 9 ( SIS) s.. B OEKBESCHOUWI ' , vOOR 1 8...",84.53,79.03,34.03,75.44,35.5,0.281618,NaN,dbnl,478466,564183


In [70]:
meta_data4.head(1)

,ti_id,titel,jaar,druk
0,_vad003177601,Vaderlandsche letteroefeningen. Jaargang 1776,1776,1ste druk


In [71]:
meta_data4['year'] = meta_data4['jaar']
meta_data4['identifier'] = meta_data4['ti_id'].astype(str)
meta_data4 = meta_data4[['identifier', 'year']]

In [72]:
df_4_year = pd.merge(left=df_4, right=meta_data4, on='identifier')
df_4_year.head(1)

,identifier,gt text,ocr text,CER,WER,WER (order independent),dictionary lookup gt,dictionary lookup ocr,jaccard_coefficient,levenshtein_dist_normalized,source,word count gt,word count ocr,year
0,_vad003182501,Boekbeschouwing. De gemoedsgesteldheid van Dav...,"SEM1N 9 ( SIS) s.. B OEKBESCHOUWI ' , vOOR 1 8...",84.53,79.03,34.03,75.44,35.5,0.281618,NaN,dbnl,478466,564183,1825


In [73]:
df_1_year.head(1)

,identifier,gt text,ocr text,CER,WER,WER (order independent),dictionary lookup gt,dictionary lookup ocr,jaccard_coefficient,levenshtein_dist_normalized,source,word count gt,word count ocr,year
0,ddd:010728187:mpeg21:a0005,"NEDERLANDEN. Valencijn, den 23 July. Sedert de...","NEDERLANDEN. Vakncijn, den j -> July. Scdeit d...",34.66,49.34,46.1,87.99,51.68,0.254021,0.22,Meertens,1968,1791,16--


In [74]:
df_year = pd.concat([df_1_year, df_2_year, df_3_year, df_4_year, df_5_year])

In [75]:
#length should be len(df)-2000
print(len(df) - 2000)
print(len(df_year))

38798
38798


In [76]:
df_year['century'] = df_year['year'].apply(lambda x: str(x)[:2] + '00s')

In [77]:
print(df_year.head())

                   identifier  \
0  ddd:010728187:mpeg21:a0005   
1  ddd:010728188:mpeg21:a0001   
2  ddd:010728188:mpeg21:a0002   
3  ddd:010728188:mpeg21:a0003   
4  ddd:010731495:mpeg21:a0001   

                                             gt text  \
0  NEDERLANDEN. Valencijn, den 23 July. Sedert de...   
1  VRANCKRYCK. PArijs, den 12 Augusti. Den Hertog...   
2  DUYTSLANDT, &amp;c. Straetsburg, den 12 Aug. G...   
3  NEDERLANDEN. Uyt het Leger onder sijne Hooghey...   
4  Ordinarise Middel-weeckse Courante, No. 30. 16...   

                                            ocr text    CER    WER  \
0  NEDERLANDEN. Vakncijn, den j -> July. Scdeit d...  34.66  49.34   
1  VRANKRYK. PÜAriif , den 12 Den Hertog van Sund...  11.86  37.43   
2  DUYTSLANDT, & C. Strertil-H,?, den ft Gifteren...  31.87  70.23   
3  NEDERLANDEN. iVl.t het. Leger tndtr fi'yne Hoo...  17.50  51.32   
4  Uyt Madrid , den 30 Jury. Stère .ïliajeflept t...  49.03  84.17   

   WER (order independent)  dictionary look

In [78]:
print(df_year['century'].unique())

['1600s' '1700s' '1900s' '1800s']


In [79]:
#df_17th = df_year[df_year['century'] == '1600s']
#df_18th = df_year[df_year['century'] == '1700s']
#df_19th = df_year[df_year['century'] == '1800s']
#df_20th = df_year[df_year['century'] == '1900s']

In [80]:
#df_17th.head

In [81]:
df_year.to_csv('df_centuries.csv', index=True)